In [1]:
# Let's start by importing the file
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler # do not use the function Normalise() - it does something entirely different
from sklearn.preprocessing import StandardScaler
import math

# We can read from an online URL
fifadata = pd.read_csv('fifa21_train.csv')

fifadata.head()

,ID,Name,Age,Nationality,Club,BP,Position,Team & Contract,Height,Weight,...,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,FC Lausanne-Sport 2015 ~ 2020,"5'9""",161lbs,...,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"Beijing Sinobo Guoan FC Dec 31, 2020 On Loan","6'0""",159lbs,...,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,Al Hilal 2019 ~ 2022,"5'4""",134lbs,...,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,Swansea City 2016 ~ 2021,"5'10""",152lbs,...,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,US Orléans Loiret Football 2018 ~ 2021,"5'11""",150lbs,...,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65


## Cleaning Data

In [2]:
#count the number of rows and columns
fifadata.shape

(11701, 101)

In [3]:
#convert the column names to lower case
fifadata.columns=fifadata.columns.str.lower()
fifadata

,id,name,age,nationality,club,bp,position,team & contract,height,weight,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,FC Lausanne-Sport 2015 ~ 2020,"5'9""",161lbs,...,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"Beijing Sinobo Guoan FC Dec 31, 2020 On Loan","6'0""",159lbs,...,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,Al Hilal 2019 ~ 2022,"5'4""",134lbs,...,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,Swansea City 2016 ~ 2021,"5'10""",152lbs,...,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,US Orléans Loiret Football 2018 ~ 2021,"5'11""",150lbs,...,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,232504,B. Böðvarsson,25,Iceland,Jagiellonia Białystok,LB,LB,Jagiellonia Białystok 2018 ~ 2021,"6'1""",168lbs,...,60+2,60+2,63+2,63+2,61+2,61+2,61+2,63+2,16+2,65
11697,214680,G. Gallon,27,France,ESTAC Troyes,GK,GK,ESTAC Troyes 2019 ~ 2022,"6'1""",174lbs,...,26+2,26+2,25+2,24+2,26+2,26+2,26+2,24+2,69+2,70
11698,221489,J. Flores,22,Chile,CD Antofagasta,RM,LM CAM RM,CD Antofagasta 2019 ~ 2024,"5'6""",143lbs,...,44+2,44+2,49+2,45+2,35+2,35+2,35+2,45+2,17+2,67
11699,146717,Anderson Silva,26,Brazil,Barnsley,CM,NaN,Barnsley 2010,"6'2""",179lbs,...,68+0,68+0,66+0,64+0,60+0,60+0,60+0,64+0,25+0,68


In [4]:
#White spaces in column names should be replaced by _

fifadata.columns = fifadata.columns.str.replace(' ', '_') 
fifadata.head()

,id,name,age,nationality,club,bp,position,team_&_contract,height,weight,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,FC Lausanne-Sport 2015 ~ 2020,"5'9""",161lbs,...,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"Beijing Sinobo Guoan FC Dec 31, 2020 On Loan","6'0""",159lbs,...,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,Al Hilal 2019 ~ 2022,"5'4""",134lbs,...,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,Swansea City 2016 ~ 2021,"5'10""",152lbs,...,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,US Orléans Loiret Football 2018 ~ 2021,"5'11""",150lbs,...,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65


In [5]:
fifadata['w/f'] = fifadata['w/f'].str.replace('★', '')
fifadata['sm'] = fifadata['sm'].str.replace('★', '')
fifadata['ir'] = fifadata['ir'].str.replace('★', '')

fifadata['w/f'] = pd.to_numeric(fifadata['w/f'], errors='coerce')
fifadata['sm'] = pd.to_numeric(fifadata['sm'], errors='coerce')
fifadata['ir'] = pd.to_numeric(fifadata['ir'], errors='coerce')

In [6]:
#check for null values
fifadata.isna().sum() #check for null values

id              0
name            0
age             0
nationality     0
club           18
               ..
cb              0
rcb             0
rb              0
gk              0
ova             0
Length: 101, dtype: int64

In [7]:
#we are adding a new column for contract by calculating the length of the contract
def calculate_contract_length(contract):
    try:
        start_year, end_year = map(int, contract.split(' ~ '))
        return end_year - start_year
    except (ValueError, TypeError):
        return 0
fifadata['contract_length'] = fifadata['contract'].apply(calculate_contract_length)
fifadata['contract_length']

0        5
1        0
2        3
3        5
4        3
        ..
11696    3
11697    3
11698    5
11699    0
11700    1
Name: contract_length, Length: 11701, dtype: int64

In [8]:
#droping the columns: team_&_contract and loan data end and position
fifadata=fifadata.drop(columns='loan_date_end')
fifadata=fifadata.drop(columns='team_&_contract')
fifadata=fifadata.drop(columns='position')

fifadata.shape
fifadata.isna().sum() #check for null values


id                  0
name                0
age                 0
nationality         0
club               18
                   ..
rcb                 0
rb                  0
gk                  0
ova                 0
contract_length     0
Length: 99, dtype: int64

In [9]:
#dropping all the null values
fifadata=fifadata.dropna() 
fifadata.isnull().sum().sum() # we dropped almost 500 rows (4% of data)
fifadata 

,id,name,age,nationality,club,bp,height,weight,foot,growth,...,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova,contract_length
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,"5'9""",161lbs,Right,1,...,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64,5
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,"6'0""",159lbs,Right,0,...,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77,0
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,"5'4""",134lbs,Right,0,...,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80,3
3,233796,J. Evans,22,Wales,Swansea City,CDM,"5'10""",152lbs,Right,13,...,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59,5
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,"5'11""",150lbs,Right,8,...,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11695,196848,I. Ávila,29,Peru,Melgar FBC,ST,"5'7""",154lbs,Right,0,...,45+2,46+2,44+2,43+2,43+2,43+2,44+2,19+2,65,2
11696,232504,B. Böðvarsson,25,Iceland,Jagiellonia Białystok,LB,"6'1""",168lbs,Left,5,...,60+2,63+2,63+2,61+2,61+2,61+2,63+2,16+2,65,3
11697,214680,G. Gallon,27,France,ESTAC Troyes,GK,"6'1""",174lbs,Right,4,...,26+2,25+2,24+2,26+2,26+2,26+2,24+2,69+2,70,3
11698,221489,J. Flores,22,Chile,CD Antofagasta,RM,"5'6""",143lbs,Right,8,...,44+2,49+2,45+2,35+2,35+2,35+2,45+2,17+2,67,5


In [10]:
fifadata.shape
fifadata['weight'].dtypes

dtype('O')

In [11]:
#removing lbs from weight column
#fifadata=fifadata['weight'].str.replace("lbs",'')
fifadata['weight']=fifadata['weight'].str.replace('lbs', '')


In [12]:
#converting it into numeric
fifadata['weight'] = pd.to_numeric(fifadata['weight'], errors='coerce')
fifadata['weight']

0        161
1        159
2        134
3        152
4        150
        ... 
11695    154
11696    168
11697    174
11698    143
11700    143
Name: weight, Length: 11411, dtype: int64

In [13]:
fifadata['weight'].dtypes


dtype('int64')

In [14]:
fifadata.dtypes

id                  int64
name               object
age                 int64
nationality        object
club               object
                    ...  
rcb                object
rb                 object
gk                 object
ova                 int64
contract_length     int64
Length: 99, dtype: object

In [15]:
# converting height into cm
fifadata['height'] = (fifadata['height']
 .str.extract(r'(\d+)\'\s*(\d+)"')
 .astype(float)
 .mul([12*2.54, 2.54])
 .sum(axis=1)
 )
fifadata['height']

#we got the code from the below link
#https://stackoverflow.com/questions/73007247/converting-inches-to-cm-on-series

0        175.26
1        182.88
2        162.56
3        177.80
4        180.34
          ...  
11695    170.18
11696    185.42
11697    185.42
11698    167.64
11700    170.18
Name: height, Length: 11411, dtype: float64

In [16]:
#calculating the ratings for the mentioned columns
columns_to_process = ['ls','st','rs','lw','lf','cf','rf','rw','lam','cam','ram',
                      'lm','lcm','cm','rcm','rm','lwb','ldm','cdm','rdm','rwb','lb','lcb','cb','rcb','rb','gk']
def process_calculation(cell):
    try:
        result = eval(cell)
        return result
    except:
        return None
for column in columns_to_process:
    fifadata[column] = fifadata[column].apply(process_calculation)
    
fifadata

,id,name,age,nationality,club,bp,height,weight,foot,growth,...,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova,contract_length
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,175.26,161,Right,1,...,60,60,59,55,55,55,59,16,64,5
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,182.88,159,Right,0,...,55,59,55,50,50,50,55,20,77,0
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,162.56,134,Right,0,...,58,61,55,43,43,43,55,14,80,3
3,233796,J. Evans,22,Wales,Swansea City,CDM,177.80,152,Right,13,...,60,58,59,60,60,60,59,16,59,5
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,180.34,150,Right,8,...,66,66,65,63,63,63,65,17,65,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11695,196848,I. Ávila,29,Peru,Melgar FBC,ST,170.18,154,Right,0,...,47,48,46,45,45,45,46,21,65,2
11696,232504,B. Böðvarsson,25,Iceland,Jagiellonia Białystok,LB,185.42,168,Left,5,...,62,65,65,63,63,63,65,18,65,3
11697,214680,G. Gallon,27,France,ESTAC Troyes,GK,185.42,174,Right,4,...,28,27,26,28,28,28,26,71,70,3
11698,221489,J. Flores,22,Chile,CD Antofagasta,RM,167.64,143,Right,8,...,46,51,47,37,37,37,47,19,67,5


In [17]:
# Assuming df is your DataFrame and 'market_value', 'weekly_wage', and 'release_clause' are the columns containing the values
fifadata['value'] = fifadata['value'].str.replace('[MKB€]', '', regex=True)
fifadata['wage'] = fifadata['wage'].str.replace('[MKB€]', '', regex=True)
fifadata['release_clause'] = fifadata['release_clause'].str.replace('[MKB€]', '', regex=True)

# Create a new column to identify the unit of each value (M for million, K for thousand)
fifadata['unit'] = fifadata['value'].str[-1]

# Convert values to numeric
fifadata['value'] = pd.to_numeric(fifadata['value'], errors='coerce')
fifadata['wage'] = pd.to_numeric(fifadata['wage'], errors='coerce')
fifadata['release_clause'] = pd.to_numeric(fifadata['release_clause'], errors='coerce')

# Multiply values based on the unit column
fifadata['value'] = fifadata.apply(lambda x: x['value'] * 1000000 if x['unit'] == 'M' else x['value'] * 1000, axis=1)
fifadata['wage'] = fifadata.apply(lambda x: x['wage'] * 1000000 if x['unit'] == 'M' else x['wage'] * 1000, axis=1)
fifadata['release_clause'] = fifadata.apply(lambda x: x['release_clause'] * 1000000 if x['unit'] == 'M' else x['release_clause'] * 1000, axis=1)

# Remove the 'unit' column if no longer needed
fifadata.drop('unit', axis=1, inplace=True)
fifadata['value']


0        525000.0
1          8500.0
2          9000.0
3        275000.0
4        725000.0
           ...   
11695    550000.0
11696    625000.0
11697      1600.0
11698      1100.0
11700    700000.0
Name: value, Length: 11411, dtype: float64

In [18]:
#we separate numerical columns and categorial columns
X_numerical = fifadata.select_dtypes(np.number)
X_categorical = fifadata.select_dtypes(object)
column_dropped= ['rb','cb', 'lb','lf','lam','sm','w/f','rwb','lwb','cdm','cam','rm','lm','rw','lw','cf','gk_diving',
                 'gk_handling','gk_kicking','gk_positioning','gk_reflexes',
                 'ls', 'rs', 'rf', 'ram', 'lcm', 'rcm', 'rm', 'ldm', 'rdm', 'lcb', 'rcb',
                 'attacking', 'crossing', 'finishing', 'heading_accuracy', 
                 'short_passing', 'volleys', 'skill', 'curve', 'fk_accuracy', 'long_passing', 
                 'ball_control', 'movement', 'acceleration', 'sprint_speed', 'agility', 'balance', 
                 'shot_power', 'jumping', 'stamina', 'strength',
                 'long_shots', 'mentality', 'aggression', 'vision', 'composure', 'defending', 
                 'marking', 'standing_tackle', 'sliding_tackle', 'growth','ova' ]
X_numerical=X_numerical.drop(columns=column_dropped)
X_numerical

,id,age,height,weight,value,wage,release_clause,dribbling,reactions,power,...,pac,sho,pas,dri,def,phy,st,cm,gk,contract_length
0,184383,26,175.26,161,525000.0,4000,801000.0,61,66,306,...,69,51,63,63,51,60,59,64,16,5
1,188044,30,182.88,159,8500.0,23000,0.0,83,75,372,...,83,75,68,82,33,71,77,70,20,0
2,184431,33,162.56,134,9000.0,49000,15300.0,85,78,308,...,80,77,78,86,27,56,75,76,14,3
3,233796,22,177.80,152,275000.0,4000,694000.0,54,55,277,...,57,44,54,57,57,60,52,58,16,5
4,234799,23,180.34,150,725000.0,2000,1400.0,64,62,280,...,66,44,60,64,60,66,58,65,17,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11695,196848,29,170.18,154,550000.0,500000,1200.0,62,71,303,...,66,68,47,66,33,57,65,55,21,2
11696,232504,25,185.42,168,625000.0,3000,891000.0,57,53,273,...,75,28,56,59,60,69,50,58,18,3
11697,214680,27,185.42,174,1600.0,4000,2900.0,17,65,217,...,72,67,66,72,40,70,30,27,71,3
11698,221489,22,167.64,143,1100.0,2000,1700.0,71,62,278,...,76,60,60,72,26,43,64,60,19,5


In [19]:
#For the numerical variables, check the multicollinearity between the features. 

correlations_matrix2 = X_numerical.corr()

fig, ax=plt.subplots(figsize=(100,100))
sns.heatmap(correlations_matrix2, annot=True)
plt.show()

## Scalling

In [20]:
# scalling
transformer = MinMaxScaler().fit(X_numerical)
X_norm = transformer.transform(X_numerical)
print(X_norm.shape)
X_num_scale = pd.DataFrame(X_norm, columns=X_numerical.columns)
X_num_scale.head()

(11411, 27)


,id,age,height,weight,value,wage,release_clause,dribbling,reactions,power,...,pac,sho,pas,dri,def,phy,st,cm,gk,contract_length
0,0.711611,0.370370,0.40,0.349206,0.538462,0.004211,0.801802,0.615385,0.591549,0.556270,...,0.614286,0.447368,0.558824,0.522388,0.506494,0.515625,0.565789,0.630137,0.0625,0.208333
1,0.725743,0.518519,0.55,0.333333,0.008718,0.024211,0.000000,0.857143,0.718310,0.768489,...,0.814286,0.763158,0.632353,0.805970,0.272727,0.687500,0.802632,0.712329,0.1125,0.000000
2,0.711796,0.629630,0.15,0.134921,0.009231,0.051579,0.015315,0.879121,0.760563,0.562701,...,0.771429,0.789474,0.779412,0.865672,0.194805,0.453125,0.776316,0.794521,0.0375,0.125000
3,0.902358,0.222222,0.45,0.277778,0.282051,0.004211,0.694695,0.538462,0.436620,0.463023,...,0.442857,0.355263,0.426471,0.432836,0.584416,0.515625,0.473684,0.547945,0.0625,0.208333
4,0.906230,0.259259,0.50,0.261905,0.743590,0.002105,0.001401,0.648352,0.535211,0.472669,...,0.571429,0.355263,0.514706,0.537313,0.623377,0.609375,0.552632,0.643836,0.0750,0.125000


In [21]:
#getting the categorial features
X_categorical

,name,nationality,club,bp,foot,joined,contract,a/w,d/w,hits
0,A. Pasche,Switzerland,FC Lausanne-Sport,CM,Right,"Jul 1, 2015",2015 ~ 2020,High,Medium,3
1,Alan Carvalho,China PR,Beijing Sinobo Guoan FC,ST,Right,"Jan 16, 2015","Dec 31, 2020 On Loan",High,Low,44
2,S. Giovinco,Italy,Al Hilal,CAM,Right,"Jan 31, 2019",2019 ~ 2022,High,Medium,73
3,J. Evans,Wales,Swansea City,CDM,Right,"Jul 1, 2016",2016 ~ 2021,Medium,Medium,7
4,Y. Demoncy,France,US Orléans Loiret Football,CDM,Right,"Jul 1, 2018",2018 ~ 2021,Low,Medium,4
...,...,...,...,...,...,...,...,...,...,...
11695,I. Ávila,Peru,Melgar FBC,ST,Right,"Jul 1, 2019",2019 ~ 2021,High,Low,4
11696,B. Böðvarsson,Iceland,Jagiellonia Białystok,LB,Left,"Feb 8, 2018",2018 ~ 2021,Medium,Medium,3
11697,G. Gallon,France,ESTAC Troyes,GK,Right,"Jul 1, 2019",2019 ~ 2022,Medium,Medium,3
11698,J. Flores,Chile,CD Antofagasta,RM,Right,"Jan 23, 2019",2019 ~ 2024,High,Medium,6


In [22]:
#dropping 'joined','name', and 'contract' because it does not add any value to our model
column_dropped= ['name','club','nationality','joined','contract','hits','bp']
X_categorical=X_categorical.drop(columns=column_dropped)
X_categorical

,foot,a/w,d/w
0,Right,High,Medium
1,Right,High,Low
2,Right,High,Medium
3,Right,Medium,Medium
4,Right,Low,Medium
...,...,...,...
11695,Right,High,Low
11696,Left,Medium,Medium
11697,Right,Medium,Medium
11698,Right,High,Medium


## Encoding categorical variables

In [23]:
#one hot encoding is a way to turn categorical variables into multiple numerical columns


from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first').fit(X_categorical)
print(encoder.categories_)


# Get the feature names generated by the encoder
column_names = encoder.get_feature_names_out()
display(column_names)


encoded = encoder.transform(X_categorical).toarray()

onehot_encoded = pd.DataFrame(encoded,columns=column_names)
onehot_encoded.head(20)

[array(['Left', 'Right'], dtype=object), array(['High', 'Low', 'Medium'], dtype=object), array(['High', 'Low', 'Medium'], dtype=object)]


array(['foot_Right', 'a/w_Low', 'a/w_Medium', 'd/w_Low', 'd/w_Medium'],
      dtype=object)

,foot_Right,a/w_Low,a/w_Medium,d/w_Low,d/w_Medium
0,1.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,1.0
3,1.0,0.0,1.0,0.0,1.0
4,1.0,1.0,0.0,0.0,1.0
5,1.0,0.0,0.0,1.0,0.0
6,1.0,0.0,1.0,0.0,0.0
7,0.0,0.0,1.0,0.0,1.0
8,1.0,0.0,1.0,0.0,1.0
9,0.0,0.0,1.0,0.0,1.0


## Train Test Split And Modelling

In [24]:
X=pd.concat([X_num_scale, onehot_encoded], axis=1) 

X

,id,age,height,weight,value,wage,release_clause,dribbling,reactions,power,...,phy,st,cm,gk,contract_length,foot_Right,a/w_Low,a/w_Medium,d/w_Low,d/w_Medium
0,0.711611,0.370370,0.40,0.349206,0.538462,0.004211,0.801802,0.615385,0.591549,0.556270,...,0.515625,0.565789,0.630137,0.0625,0.208333,1.0,0.0,0.0,0.0,1.0
1,0.725743,0.518519,0.55,0.333333,0.008718,0.024211,0.000000,0.857143,0.718310,0.768489,...,0.687500,0.802632,0.712329,0.1125,0.000000,1.0,0.0,0.0,1.0,0.0
2,0.711796,0.629630,0.15,0.134921,0.009231,0.051579,0.015315,0.879121,0.760563,0.562701,...,0.453125,0.776316,0.794521,0.0375,0.125000,1.0,0.0,0.0,0.0,1.0
3,0.902358,0.222222,0.45,0.277778,0.282051,0.004211,0.694695,0.538462,0.436620,0.463023,...,0.515625,0.473684,0.547945,0.0625,0.208333,1.0,0.0,1.0,0.0,1.0
4,0.906230,0.259259,0.50,0.261905,0.743590,0.002105,0.001401,0.648352,0.535211,0.472669,...,0.609375,0.552632,0.643836,0.0750,0.125000,1.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11406,0.759729,0.481481,0.30,0.293651,0.564103,0.526316,0.001201,0.626374,0.661972,0.546624,...,0.468750,0.644737,0.506849,0.1250,0.083333,1.0,0.0,0.0,1.0,0.0
11407,0.897371,0.333333,0.60,0.404762,0.641026,0.003158,0.891892,0.571429,0.408451,0.450161,...,0.656250,0.447368,0.547945,0.0875,0.125000,0.0,0.0,1.0,0.0,1.0
11408,0.828565,0.407407,0.60,0.452381,0.001641,0.004211,0.002903,0.131868,0.577465,0.270096,...,0.671875,0.184211,0.123288,0.7500,0.125000,1.0,0.0,1.0,0.0,1.0
11409,0.854850,0.222222,0.25,0.206349,0.001128,0.002105,0.001702,0.725275,0.535211,0.466238,...,0.250000,0.631579,0.575342,0.1000,0.208333,1.0,0.0,0.0,0.0,1.0


In [25]:
y=fifadata['ova']
y

0        64
1        77
2        80
3        59
4        65
         ..
11695    65
11696    65
11697    70
11698    67
11700    64
Name: ova, Length: 11411, dtype: int64

In [26]:
# import library
from sklearn.model_selection import train_test_split as tts

X_train, X_test, y_train, y_test=tts(X, y, test_size=.2)

#import library
from sklearn.linear_model import LinearRegression as linreg

lm = linreg()    # model
lm.fit(X_train, y_train)   # model training
y_pred = lm.predict(X_test)   # model prediction



In [27]:
#R2
print ('train R2: {} -- test R2: {}'.format(lm.score(X_train, y_train),
                                            lm.score(X_test, y_test)))

train R2: 0.8961077077797769 -- test R2: 0.8864044654356922


In [28]:
#mean_squared_error mse
from sklearn.metrics import mean_squared_error as mse


train_mse=mse(lm.predict(X_train), y_train)
test_mse=mse(y_pred, y_test)

print ('train MSE: {} -- test MSE: {}'.format(train_mse, test_mse))

train MSE: 4.910663307183254 -- test MSE: 5.084210523220201


In [29]:
#RMSE
print ('train RMSE: {} -- test RMSE: {}'.format(train_mse**.5, test_mse**.5))

train RMSE: 2.2160016487320706 -- test RMSE: 2.254819399246911


In [30]:
#mae
from sklearn.metrics import mean_absolute_error as mae

train_mae=mae(lm.predict(X_train), y_train)
test_mae=mae(y_pred, y_test)

print ('train MAE: {} -- test MAE: {}'.format(train_mae, test_mae))

train MAE: 1.7283598542944785 -- test MAE: 1.7512490418309241


In [ ]:
#O#R2 score is quite good. In average the predicted OVA is 1.75 deviated from the actual value 

## Function For Cleaning The data

In [31]:
#reading the old and new validated data frame

fifadata1 = pd.read_csv('fifa21_train.csv')
fifadata_p = pd.read_csv('fifa21_validate.csv')

#comparing both the dataframes
list(fifadata1.columns) == list(fifadata_p.columns)



True

In [32]:
def clean_data(df):
    
    df.columns=df.columns.str.lower() #converting into lower case
    df.columns = df.columns.str.replace(' ', '_') #replace white space with _
    df['contract_length'] = df['contract'].apply(calculate_contract_length) #calculate the contract length 
    df=df.drop(columns='loan_date_end') #drop the columns loan_date_end,team_&_contract,position
    df=df.drop(columns='team_&_contract')
    df=df.drop(columns='position')

    df['w/f'] = df['w/f'].str.replace('★', '')#replacing the star with blank n converting it into numeric
    df['sm'] = df['sm'].str.replace('★', '')
    df['ir'] = df['ir'].str.replace('★', '')

    df['w/f'] = pd.to_numeric(df['w/f'], errors='coerce')
    df['sm'] = pd.to_numeric(df['sm'], errors='coerce')
    df['ir'] = pd.to_numeric(df['ir'], errors='coerce')

    df=df.dropna() #dropping all the null values
    df['weight']=df['weight'].str.replace('lbs', '') #replace lbs with blank
    df['weight'] = pd.to_numeric(df['weight'], errors='coerce') #convert weight into numeric
    df['height'] = (df['height'].str.extract(r'(\d+)\'\s*(\d+)"').astype(float).mul([12*2.54, 2.54]).sum(axis=1)) 
                                                                                     # we are converting height from feet into cm
    for column in columns_to_process:
         df[column] = df[column].apply(process_calculation)

    df['value'] = df['value'].str.replace('[MKB€]', '', regex=True) # replacing MKB€ with blank
    df['wage'] = df['wage'].str.replace('[MKB€]', '', regex=True)
    df['release_clause'] = df['release_clause'].str.replace('[MKB€]', '', regex=True)

# Create a new column to identify the unit of each value (M for million, K for thousand)
    df['unit'] = df['value'].str[-1]

# Convert values to numeric
    df['value'] = pd.to_numeric(df['value'], errors='coerce') # converting columns into numeric
    df['wage'] = pd.to_numeric(df['wage'], errors='coerce')
    df['release_clause'] = pd.to_numeric(df['release_clause'], errors='coerce')

# Multiply values based on the unit column
    df['value'] = df.apply(lambda x: x['value'] * 1000000 if x['unit'] == 'M' else x['value'] * 1000, axis=1) # calculating the 
                                                                                    #columns based on units(M,K,€) 
    df['wage'] = df.apply(lambda x: x['wage'] * 1000000 if x['unit'] == 'M' else x['wage'] * 1000, axis=1)
    df['release_clause'] = df.apply(lambda x: x['release_clause'] * 1000000 if x['unit'] == 'M' else x['release_clause'] * 1000, axis=1)

# Removing the 'unit' column if no longer needed
    df.drop('unit', axis=1, inplace=True)


    column_dropped= ['rb','cb', 'lb','lf','lam','sm','w/f','rwb','lwb','cdm','cam','rm','lm','rw','lw','cf','gk_diving',
                 'gk_handling','gk_kicking','gk_positioning','gk_reflexes',
                 'ls', 'rs', 'rf', 'ram', 'lcm', 'rcm', 'rm', 'ldm', 'rdm', 'lcb', 'rcb',
                 'attacking', 'crossing', 'finishing', 'heading_accuracy', 
                 'short_passing', 'volleys', 'skill', 'curve', 'fk_accuracy', 'long_passing', 
                 'ball_control', 'movement', 'acceleration', 'sprint_speed', 'agility', 'balance', 
                 'shot_power', 'jumping', 'stamina', 'strength',
                 'long_shots', 'mentality', 'aggression', 'vision', 'composure', 'defending', 
                 'marking', 'standing_tackle', 'sliding_tackle', 'growth','name','club',
                 'nationality','joined','contract','hits','bp']
    df=df.drop(columns=column_dropped)

    return df

In [33]:
fifadata_p = clean_data(fifadata_p)
fifadata_p

,id,age,height,weight,foot,value,wage,release_clause,dribbling,reactions,...,sho,pas,dri,def,phy,st,cm,gk,ova,contract_length
0,219461,23,187.96,194,Right,975000.0,5000,0.0,44,63,...,30,50,50,66,74,50,53,18,67,0
1,221896,22,172.72,154,Right,1200.0,3000,2200.0,76,64,...,67,62,74,24,57,66,63,19,68,0
2,247428,19,187.96,176,Right,120000.0,500000,249000.0,6,41,...,52,53,55,26,53,20,20,55,54,2
3,255120,16,190.50,170,Right,160000.0,500000,464000.0,51,48,...,34,47,52,53,59,48,51,13,55,2
4,215556,24,187.96,170,Right,2300.0,13000,4300.0,71,64,...,62,68,70,69,71,65,72,20,70,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,239074,21,172.72,143,Left,325000.0,1000,731000.0,51,50,...,28,46,55,53,57,46,50,17,60,3
1995,241223,21,195.58,176,Right,190000.0,700000,285000.0,9,59,...,55,57,62,30,54,24,26,60,59,4
1996,210930,27,170.18,146,Left,8000.0,9000,12000.0,77,73,...,69,78,77,39,60,69,76,20,76,5
1997,162993,34,180.34,176,Right,140000.0,4000,245000.0,56,61,...,47,58,58,61,69,57,61,16,63,1


In [34]:
#categorical features and numerical ones are going to be treated differently
X_num1 = fifadata_p.select_dtypes(include = np.number)
X_cat1 = fifadata_p.select_dtypes(include = object)

X_num1=X_num1.drop(columns=['ova'])
X_num1

,id,age,height,weight,value,wage,release_clause,dribbling,reactions,power,...,pac,sho,pas,dri,def,phy,st,cm,gk,contract_length
0,219461,23,187.96,194,975000.0,5000,0.0,44,63,288,...,64,30,50,50,66,74,50,53,18,0
1,221896,22,172.72,154,1200.0,3000,2200.0,76,64,323,...,74,67,62,74,24,57,66,63,19,0
2,247428,19,187.96,176,120000.0,500000,249000.0,6,41,171,...,56,52,53,55,26,53,20,20,55,2
3,255120,16,190.50,170,160000.0,500000,464000.0,51,48,242,...,58,34,47,52,53,59,48,51,13,2
4,215556,24,187.96,170,2300.0,13000,4300.0,71,64,337,...,67,62,68,70,69,71,65,72,20,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,239074,21,172.72,143,325000.0,1000,731000.0,51,50,245,...,76,28,46,55,53,57,46,50,17,3
1995,241223,21,195.58,176,190000.0,700000,285000.0,9,59,177,...,60,55,57,62,30,54,24,26,60,4
1996,210930,27,170.18,146,8000.0,9000,12000.0,77,73,329,...,65,69,78,77,39,60,69,76,20,5
1997,162993,34,180.34,176,140000.0,4000,245000.0,56,61,300,...,53,47,58,58,61,69,57,61,16,1


In [35]:
X_cat1

,foot,a/w,d/w
0,Right,Low,High
1,Right,High,Low
2,Right,Medium,Medium
3,Right,Medium,Medium
4,Right,Medium,Medium
...,...,...,...
1994,Left,High,Medium
1995,Right,Medium,Medium
1996,Left,High,Medium
1997,Right,Medium,Medium


In [36]:
X_for_p_normalized = transformer.transform(X_num1)
X_for_p_normalized = pd.DataFrame(X_for_p_normalized,columns=X_num1.columns)
X_for_p_normalized

,id,age,height,weight,value,wage,release_clause,dribbling,reactions,power,...,pac,sho,pas,dri,def,phy,st,cm,gk,contract_length
0,0.847021,0.259259,0.65,0.611111,1.000000,0.005263,0.000000,0.428571,0.549296,0.498392,...,0.542857,0.171053,0.367647,0.328358,0.701299,0.734375,0.447368,0.479452,0.0875,0.000000
1,0.856421,0.222222,0.35,0.293651,0.001231,0.003158,0.002202,0.780220,0.563380,0.610932,...,0.685714,0.657895,0.544118,0.686567,0.155844,0.468750,0.657895,0.616438,0.1000,0.000000
2,0.954981,0.111111,0.65,0.468254,0.123077,0.526316,0.249249,0.010989,0.239437,0.122186,...,0.428571,0.460526,0.411765,0.402985,0.181818,0.406250,0.052632,0.027397,0.5500,0.083333
3,0.984675,0.000000,0.70,0.420635,0.164103,0.526316,0.464464,0.505495,0.338028,0.350482,...,0.457143,0.223684,0.323529,0.358209,0.532468,0.500000,0.421053,0.452055,0.0250,0.083333
4,0.831947,0.296296,0.65,0.420635,0.002359,0.013684,0.004304,0.725275,0.563380,0.655949,...,0.585714,0.592105,0.632353,0.626866,0.740260,0.687500,0.644737,0.739726,0.1125,0.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1942,0.922733,0.185185,0.35,0.206349,0.333333,0.001053,0.731732,0.505495,0.366197,0.360129,...,0.714286,0.144737,0.308824,0.402985,0.532468,0.468750,0.394737,0.438356,0.0750,0.125000
1943,0.931028,0.185185,0.80,0.468254,0.194872,0.736842,0.285285,0.043956,0.492958,0.141479,...,0.485714,0.500000,0.470588,0.507463,0.233766,0.421875,0.105263,0.109589,0.6125,0.166667
1944,0.814089,0.407407,0.30,0.230159,0.008205,0.009474,0.012012,0.791209,0.690141,0.630225,...,0.557143,0.684211,0.779412,0.731343,0.350649,0.515625,0.697368,0.794521,0.1125,0.208333
1945,0.629039,0.666667,0.50,0.468254,0.143590,0.004211,0.245245,0.560440,0.521127,0.536977,...,0.385714,0.394737,0.485294,0.447761,0.636364,0.656250,0.539474,0.589041,0.0625,0.041667


In [37]:
# Get the feature names generated by the encoder
encoded = encoder.transform(X_cat1).toarray()

onehot_encoded = pd.DataFrame(encoded,columns=column_names)
onehot_encoded.head(20)

,foot_Right,a/w_Low,a/w_Medium,d/w_Low,d/w_Medium
0,1.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0
2,1.0,0.0,1.0,0.0,1.0
3,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,1.0,0.0,1.0
5,1.0,0.0,0.0,0.0,0.0
6,1.0,0.0,0.0,0.0,1.0
7,0.0,0.0,1.0,0.0,1.0
8,1.0,0.0,1.0,0.0,1.0
9,0.0,0.0,1.0,0.0,1.0


In [38]:
#merging both the numerical and categorical
X_p=pd.concat([X_for_p_normalized, onehot_encoded], axis=1) 
X_p

,id,age,height,weight,value,wage,release_clause,dribbling,reactions,power,...,phy,st,cm,gk,contract_length,foot_Right,a/w_Low,a/w_Medium,d/w_Low,d/w_Medium
0,0.847021,0.259259,0.65,0.611111,1.000000,0.005263,0.000000,0.428571,0.549296,0.498392,...,0.734375,0.447368,0.479452,0.0875,0.000000,1.0,1.0,0.0,0.0,0.0
1,0.856421,0.222222,0.35,0.293651,0.001231,0.003158,0.002202,0.780220,0.563380,0.610932,...,0.468750,0.657895,0.616438,0.1000,0.000000,1.0,0.0,0.0,1.0,0.0
2,0.954981,0.111111,0.65,0.468254,0.123077,0.526316,0.249249,0.010989,0.239437,0.122186,...,0.406250,0.052632,0.027397,0.5500,0.083333,1.0,0.0,1.0,0.0,1.0
3,0.984675,0.000000,0.70,0.420635,0.164103,0.526316,0.464464,0.505495,0.338028,0.350482,...,0.500000,0.421053,0.452055,0.0250,0.083333,1.0,0.0,1.0,0.0,1.0
4,0.831947,0.296296,0.65,0.420635,0.002359,0.013684,0.004304,0.725275,0.563380,0.655949,...,0.687500,0.644737,0.739726,0.1125,0.166667,1.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1942,0.922733,0.185185,0.35,0.206349,0.333333,0.001053,0.731732,0.505495,0.366197,0.360129,...,0.468750,0.394737,0.438356,0.0750,0.125000,0.0,0.0,0.0,0.0,1.0
1943,0.931028,0.185185,0.80,0.468254,0.194872,0.736842,0.285285,0.043956,0.492958,0.141479,...,0.421875,0.105263,0.109589,0.6125,0.166667,1.0,0.0,1.0,0.0,1.0
1944,0.814089,0.407407,0.30,0.230159,0.008205,0.009474,0.012012,0.791209,0.690141,0.630225,...,0.515625,0.697368,0.794521,0.1125,0.208333,0.0,0.0,0.0,0.0,1.0
1945,0.629039,0.666667,0.50,0.468254,0.143590,0.004211,0.245245,0.560440,0.521127,0.536977,...,0.656250,0.539474,0.589041,0.0625,0.041667,1.0,0.0,1.0,0.0,1.0


In [39]:
y_p=fifadata_p['ova']
y_p

0       67
1       68
2       54
3       55
4       70
        ..
1994    60
1995    59
1996    76
1997    63
1998    60
Name: ova, Length: 1947, dtype: int64

In [40]:
#predict and inspect results
results_for_p = lm.predict(X_p)
results_for_p

array([66.046875 , 66.3984375, 53.25     , ..., 72.9296875, 65.65625  ,
       60.3203125])

In [41]:
print("R2-score is ", lm.score(X_p,y_p))

R2-score is  0.8932338840527344


In [44]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_p, results_for_p)
print(mae)

1.7310405431432974


In [ ]:
#R2 score is quite good. In average the predicted OVA is 1.73 deviated from the actual value